<span STYLE="font-size:150%"> 
    Segment cylinder regions from microCT scans
</span>

Docker image: gnasello/slicer-env:2023-10-10 \
Latest update: 11 October 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [1]:
import psutil

# Get memory information
memoryInfo = psutil.virtual_memory()
total_memory = memoryInfo.total / (1024 ** 3)
available_memory = memoryInfo.available / (1024 ** 3)
used_memory = memoryInfo.used / (1024 ** 3)

print(f"Total Memory: {total_memory:.2f} GB")
print(f"Available Memory: {available_memory:.2f} GB")
print(f"Used Memory: {used_memory:.2f} GB")

Total Memory: 125.58 GB
Available Memory: 118.92 GB
Used Memory: 5.47 GB


In [2]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd
import trimesh

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [3]:
# this cell is tagged 'parameters'
segmented_dir = 'segmented_volumes/'
volume_file = 'microCT_volume/microCT_volume.nrrd'
scaffold_filename = segmented_dir + 'Scaffold.stl'
output_cylinders_dir = segmented_dir + 'cylinders/'

In [4]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [5]:
## mm
masterVolumeNode.GetSpacing()

(0.004659700000000001, 0.004659700000000001, 0.004659700000000001)

In [6]:
segmented_dir_path = Path(segmented_dir)

Make ```segmented_volumes/cylinders``` folder

In [7]:
output_path = Path(output_cylinders_dir)

output_path.mkdir(parents=True, exist_ok=True)

Get norebook directory name

## Adjust window/level (brightness/contrast)

Adjust the image window/level from the GUI. It is enough to select a region where the signal is present ([youtube](https://slicer.readthedocs.io/en/latest/user_guide/user_interface.html#adjusting-image-window-level)). 

After, get the Window and Level properties from the displayNode associate to the VolumeNode and apply them programmatically anytime you run again the same image.

For more information on the relationship between window/level and brightness/contrast, see [Window and Level Contrast Enhancement](http://fisica.ciens.ucv.ve/curs/dipcourse/html/one-oper/window-level/front-page.html)

In [8]:
displayNode = masterVolumeNode.GetDisplayNode()
print('displayNode.SetWindow(' + str(displayNode.GetWindow()) + ')')
print('displayNode.SetLevel(' + str(displayNode.GetLevel()) + ')')

displayNode.SetWindow(211.0)
displayNode.SetLevel(121.5)


In [9]:
# displayNode = masterVolumeNode.GetDisplayNode()
# displayNode.AutoWindowLevelOff()
# displayNode.SetWindow(12415.0)
# displayNode.SetLevel(11271.5)

# Create segmentationNode

## Create segmentation-related nodes

Create segmentation node

In [10]:
segmentationNode = ps.segmentation.segmentationNode(name='Segmentation')

Create temporary segment editor to get access to effects

In [11]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

## Make scaffold cylinders

### Compute bounding scaffold cylinder
https://docs.pyvista.org/examples/01-filter/clipping-with-surface.html

In [12]:
cylinder_mesh = trimesh.load(scaffold_filename).bounding_cylinder

In [13]:
bounds_mesh = trimesh.bounds.minimum_cylinder(cylinder_mesh)
bounds_mesh

{'height': 6.068766129935371,
 'radius': 3.1496249443317383,
 'transform': array([[ 0.07953169, -0.69649247, -0.71314301,  4.45254557],
        [ 0.03233427,  0.71683522, -0.69649247,  5.51780994],
        [ 0.99630779,  0.03233427,  0.07953169,  5.11935613],
        [ 0.        ,  0.        ,  0.        ,  1.        ]])}

In [14]:
cylinder_mesh.center_mass

array([4.45254557, 5.51780994, 5.11935613])

In [15]:
cylinder_mesh.direction

TrackedArray([ 0.71314301,  0.69649247, -0.07953169])

In [16]:
def lps_to_ras(points):
    """
    Convert a numpy array of points from LPS (Left, Posterior, Superior) to RAS (Right, Anterior, Superior) coordinates.
    See https://slicer.readthedocs.io/en/latest/user_guide/coordinate_systems.html
    
    Parameters:
    points (numpy.ndarray): A Nx3 numpy array of LPS points or a single 1x3 point.

    Returns:
    numpy.ndarray: A Nx3 numpy array of RAS points or a single 1x3 point.
    """
    from numpy import atleast_2d
    
    points = atleast_2d(points)  # Ensure the points are at least 2D
    ras_points = points.copy()
    ras_points[:, 0] = -ras_points[:, 0]  # Flip X
    ras_points[:, 1] = -ras_points[:, 1]  # Flip Y
    
    if points.shape[0] == 1:
        return ras_points[0]  # Return as 1D array if input was a single point
    return ras_points

In [17]:
center_mass_ras = lps_to_ras(cylinder_mesh.center_mass)
center_mass_ras

array([-4.45254557, -5.51780994,  5.11935613])

In [22]:
direction_ras = lps_to_ras(cylinder_mesh.direction)
direction_ras

TrackedArray([-0.71314301, -0.69649247, -0.07953169])

# `scaffold_cylinder.stl`

In [26]:
model_name = 'Scaffold-Cylinder'

In [27]:
cylinder_model = ps.model.create_hollow_cylinder(height=bounds_mesh['height'], 
                                                         radius_inner=0, radius_outer=bounds_mesh['radius'], space =5, 
                                                         center=center_mass_ras,
                                                         direction=direction_ras.tolist(),
                                                         transform=False,
                                                         nameModel=model_name, 
                                                         color=(230/255, 230/255, 77/255), 
                                                         opacity=1)

In [28]:
filename_output = model_name + '.stl'

slicer.util.exportNode(cylinder_model, output_path / filename_output)

True

Import model to segmentation node

In [29]:
segmentationNode = ps.segmentation.segmentationNode(name=model_name)

slicer.modules.segmentations.logic().ImportModelToSegmentationNode(cylinder_model, segmentationNode)

True

Export segment to labelmapNode

In [30]:
labelmap = ps.segmentation.individual_segment_to_labelmapNode(segmentName = model_name,
                                                                          segmentationNode = segmentationNode,
                                                                          volumeNode = masterVolumeNode)

In [31]:
filename_output = model_name + '.seg.nrrd'

slicer.util.exportNode(labelmap, output_path / filename_output)

True

In [32]:
#Remove the model node if it is no longer needed
slicer.mrmlScene.RemoveNode(segmentationNode)
slicer.mrmlScene.RemoveNode(cylinder_model)
slicer.mrmlScene.RemoveNode(labelmap)

# `cylinder_outer.stl`

In [37]:
model_name = 'Cylinder-Outer'

In [38]:
cylinder_model = ps.model.create_hollow_cylinder(height=bounds_mesh['height'], 
                                                         radius_inner=bounds_mesh['radius']*2/3, radius_outer=bounds_mesh['radius'], space =5, 
                                                         center=center_mass_ras,
                                                         direction=direction_ras.tolist(),
                                                         transform=False,
                                                         nameModel=model_name, 
                                                         color=(50/255, 75/255, 53/255), 
                                                         opacity=1)

In [39]:
filename_output = model_name + '.stl'

slicer.util.exportNode(cylinder_model, output_path / filename_output)

True

Import model to segmentation node

In [40]:
segmentationNode = ps.segmentation.segmentationNode(name=model_name)

slicer.modules.segmentations.logic().ImportModelToSegmentationNode(cylinder_model, segmentationNode)

True

Export segment to labelmapNode

In [41]:
labelmap = ps.segmentation.individual_segment_to_labelmapNode(segmentName = model_name,
                                                              segmentationNode = segmentationNode,
                                                              volumeNode = masterVolumeNode)

In [42]:
filename_output = model_name + '.seg.nrrd'

slicer.util.exportNode(labelmap, output_path / filename_output)

True

In [43]:
#Remove the model node if it is no longer needed
slicer.mrmlScene.RemoveNode(segmentationNode)
slicer.mrmlScene.RemoveNode(cylinder_model)
slicer.mrmlScene.RemoveNode(labelmap)

# `cylinder_middle.stl`

In [45]:
model_name = 'Cylinder-Middle'

In [46]:
cylinder_model = ps.model.create_hollow_cylinder(height=bounds_mesh['height'], 
                                                 radius_inner=bounds_mesh['radius']*1/3, radius_outer=bounds_mesh['radius']*2/3, space =5, 
                                                 center=center_mass_ras,
                                                 direction=direction_ras.tolist(),
                                                 transform=False,
                                                 nameModel=model_name, 
                                                 color=(230/255, 75/255, 53/255), 
                                                 opacity=1)

In [47]:
filename_output = model_name + '.stl'

slicer.util.exportNode(cylinder_model, output_path / filename_output)

True

Import model to segmentation node

In [48]:
segmentationNode = ps.segmentation.segmentationNode(name=model_name)

slicer.modules.segmentations.logic().ImportModelToSegmentationNode(cylinder_model, segmentationNode)

True

Export segment to labelmapNode

In [49]:
labelmap = ps.segmentation.individual_segment_to_labelmapNode(segmentName = model_name,
                                                              segmentationNode = segmentationNode,
                                                              volumeNode = masterVolumeNode)

In [50]:
filename_output = model_name + '.seg.nrrd'

slicer.util.exportNode(labelmap, output_path / filename_output)

True

In [51]:
#Remove the model node if it is no longer needed
slicer.mrmlScene.RemoveNode(segmentationNode)
slicer.mrmlScene.RemoveNode(cylinder_model)
slicer.mrmlScene.RemoveNode(labelmap)

# `cylinder_inner.stl`

In [52]:
model_name = 'Cylinder-Inner'

In [53]:
cylinder_model = ps.model.create_hollow_cylinder(height=bounds_mesh['height'], 
                                                 radius_inner=0, radius_outer=bounds_mesh['radius']*1/3, space =5, 
                                                 center=center_mass_ras,
                                                 direction=direction_ras.tolist(),
                                                 transform=False,
                                                 nameModel=model_name, 
                                                 color=(77/255, 187/255, 213/255), 
                                                 opacity=1)

In [54]:
filename_output = model_name + '.stl'

slicer.util.exportNode(cylinder_model, output_path / filename_output)

True

Import model to segmentation node

In [55]:
segmentationNode = ps.segmentation.segmentationNode(name=model_name)

slicer.modules.segmentations.logic().ImportModelToSegmentationNode(cylinder_model, segmentationNode)

True

Export segment to labelmapNode

In [56]:
labelmap = ps.segmentation.individual_segment_to_labelmapNode(segmentName = model_name,
                                                              segmentationNode = segmentationNode,
                                                              volumeNode = masterVolumeNode)

In [57]:
filename_output = model_name + '.seg.nrrd'

slicer.util.exportNode(labelmap, output_path / filename_output)

True

In [58]:
#Remove the model node if it is no longer needed
slicer.mrmlScene.RemoveNode(segmentationNode)
slicer.mrmlScene.RemoveNode(cylinder_model)
slicer.mrmlScene.RemoveNode(labelmap)